## Capstone course week 3 peer review assignment - Toronto neighbourhoods

In [37]:
#Import required libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd
import numpy as np
import sklearn

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Extract the first table from the webpage
page = urllib.request.urlopen(url).read()
soup = BeautifulSoup(page)
soup2 = soup.tbody

Scrape the contents of the table with the postal codes

In [70]:
scraped_table = []
table_rows = soup2.find_all('tr')[1:]
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    scraped_table.append(row)

#Put the contents of the table into a pandas dataframe and remove the "/n" string from each entry
df = pd.DataFrame(scraped_table, columns=["Postal Code", "Borough", "Neighbourhood"])
for col in df:
    df[col]=df[col].str.strip('\n')

In [81]:
#Drop rows where Borough is not assigned
df = df[df.Borough != "Not assigned"]
df=df.reset_index(drop=True)

In [142]:
#Merge multiple rows with identical postcode and borough values into a single row with
#multiple comma delimited neighbourhood values.

df2 = df.groupby(by=['Postal Code','Borough']).agg(lambda x: ','.join(x))
df2.reset_index(level=['Postal Code','Borough'], inplace=True)


In [143]:
print("The dimensions of the dataset are:",df.shape)

The dimensions of the dataset are: (103, 3)


In [130]:
!pip install geopy
from geopy.geocoders import Nominatim

In [144]:
#Geocoder package did not work, hence reading in csv file with lat-long of neighbourhoods
postcode = pd.read_csv("http://cocl.us/Geospatial_data")

Join the dataset to the lat-long table using Postal Code as the key. Note that the resulting
dataframe will appear different from the sample screenshot in the assignment due to grouping 
done above by Postal Code and Borough.

In [145]:
df3 = df2.join(postcode.set_index('Postal Code'), on='Postal Code')
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [149]:
#import libraries needed for work with Foursquare
import requests
import folium
from IPython.display import Image 
from IPython.core.display import HTML 


In [150]:
CLIENT_ID = 'XYZ' # your Foursquare ID
CLIENT_SECRET = 'ABC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: A41QZAAAJYGLFYS0TCQHK22SFPOKVCBVTFSTN24COKHY2BSF
CLIENT_SECRET:G0LPFB5WUQUCDJKPNW4Z1SG2H1N54WI3HP1IRAXIVIB5OSF5


#### Subset the dataset to only work with only boroughs that contain the word Toronto 

In [186]:
df4 = df3[df3['Borough'].str.contains("Toronto")]
df4.shape


(39, 5)

In [153]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top

In [157]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Create a function to explore the neighbourhoods in the dataset

In [163]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [164]:
toronto_venues = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [165]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [166]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",14,14,14,14,14,14
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [168]:
#Find number of unique venue categories in the dataset
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 194 unique categories.


#### Prepare the dataset for K-means analysis

In [171]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
tr_df = toronto_onehot[fixed_columns]

# group rows by neighbourhood and take the mean of the frequency of occurrence of each category
tr_grouped = tr_df.groupby('Neighbourhood').mean().reset_index()


In [172]:
# write a function to sort venues in the dataset in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new Toronto dataframe that displays the top 10 venues for each neighbourhood.

In [203]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = tr_grouped['Neighbourhood']

for ind in np.arange(tr_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tr_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Park,Italian Restaurant,Basketball Stadium,Bistro,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Pet Store,Bar,Burrito Place,Restaurant,Climbing Gym,Yoga Studio
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Skate Park,Burrito Place,Farmers Market,Restaurant,Comic Shop,Pizza Place,Gym / Fitness Center,Garden,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Plane,Airport Food Court
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Ramen Restaurant,Seafood Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Poke Place


#### Build the K-means model

In [204]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 6

tr_cluster = tr_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tr_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 5, 2, 1, 2, 1, 2, 1, 2, 4, 2,
       2, 2, 2, 2, 1, 3, 2, 2])

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [205]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tr_merged = df4

# merge tr_merged with the neighbourhoods_venues_sorted dataset to get cluster label and top 10 venues for each neighbourhood
tr_merged2 = tr_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tr_merged2

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Trail,Health Food Store,Pub,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place,Brewery,Juice Bar,Bookstore,Dessert Shop
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Fish & Chips Shop,Restaurant,Italian Restaurant,Burrito Place,Light Rail Station,Ice Cream Shop,Pub,Liquor Store,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Bakery,Yoga Studio,Seafood Restaurant,Sandwich Place,Brewery,Cheese Shop,Pet Store,Neighborhood
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Lawyer,Bus Line,Swim School,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Breakfast Spot,Park,Pizza Place,Gym,Sandwich Place,Department Store,Food & Drink Shop,Hotel,Creperie,Discount Store
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,2,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Health & Beauty Service,Fast Food Restaurant,Ice Cream Shop,Diner,Metro Station,Mexican Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Dessert Shop,Coffee Shop,Pizza Place,Café,Sandwich Place,Italian Restaurant,Gym,Sushi Restaurant,Farmers Market,Park
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Tennis Court,Yoga Studio,Creperie,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,2,Coffee Shop,Pub,Pizza Place,Restaurant,Light Rail Station,Bank,Liquor Store,Bagel Shop,Sports Bar,Sushi Restaurant


#### Visualize the resulting clusters

In [206]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tr_merged2['Latitude'], tr_merged2['Longitude'], tr_merged2['Neighbourhood'], tr_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [207]:
tr_merged2.sort_values(by=['Cluster Labels'], inplace=True)
tr_merged2

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Trail,Health Food Store,Pub,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner
76,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Supermarket,Pet Store,Music Venue,Bus Stop,Café,Bar,Bank,Middle Eastern Restaurant
68,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Airport,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Plane,Airport Food Court
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Trail,Playground,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Breakfast Spot,Park,Pizza Place,Gym,Sandwich Place,Department Store,Food & Drink Shop,Hotel,Creperie,Discount Store
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,1,Plaza,Café,Hotel,Park,Event Space,Skating Rink,Basketball Stadium,IT Services,Ice Cream Shop,Deli / Bodega
87,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,1,Light Rail Station,Skate Park,Burrito Place,Farmers Market,Restaurant,Comic Shop,Pizza Place,Gym / Fitness Center,Garden,Garden Center
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Fish & Chips Shop,Restaurant,Italian Restaurant,Burrito Place,Light Rail Station,Ice Cream Shop,Pub,Liquor Store,Steakhouse
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Lawyer,Bus Line,Swim School,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
84,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,2,Café,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Vegetarian / Vegan Restaurant,Fish & Chips Shop,Burrito Place,Sandwich Place,Dessert Shop


#### Observations

1. Cluster 2 is the largest cluster, and it is concentrated at the centre of Toronto. It appears to consist of neighbourhoods where F&B outlets dominate the top venues.

2. Cluster 1 is the second largest cluster, and they are on the edges of the city surrounding Cluster 2 neighbourhoods. It appears to consist of neighbourhoods where recreational spaces such as parks, gyms and playgrounds feature more heavily as top venues.